# Really messy. Needs cleaning

In [1]:
%matplotlib inline
import pandas as pd
import networkx as nx
import matplotlib as plt
import os

In [2]:
G=nx.DiGraph()

In [3]:
class node:
    
    def __init__(self,identifier,kind,name,G,parent_name=None):
        self.name=name
        self.kind=kind
        if(parent_name!=None):
            self.identifier=G.node[parent_name.strip()]["link"].identifier+":"+identifier
            self.parent=parent_name
        else:
            self.identifier=identifier
            self.parent="None"
        self.description=[]
        self.comment_list=[]
        self.other_attributes={}
        
    def add_description(self,description):
        self.description.append(description)
        
    def add_comment(self,comment):
        self.comment_list.append(comment)
        
    def add_other_attribute(self,attribute_name,attribute_value):
        try:
            self.other_attributes[attribute_name].append(attribute_value)
        except KeyError:
            self.other_attributes[attribute_name]=[attribute_value]
            
    def print_node(self):
        print("Identifier:",self.identifier,"\nName:",self.name,"\nKind:",self.kind,"\nParent:",self.parent)
        print("Description:",self.description)
        print("Comment List:",self.comment_list)
        print("Other Attributes:",self.other_attributes.keys())
        for i in self.other_attributes.keys():
            print(i,self.other_attributes[i])
            
    def insert_into_graph(self,G):
        G.add_node(self.name.strip(),link=self)
        
    def add_child(self,child,G,typ="Child"):
        G.add_edge(self.name.strip(),child.name.strip(),category=typ)
        
    def parse_graphML(self):
        string="<node id=\"%s\">\n"%(self.identifier)
        string+="<data key=\"label\">Concept</data>\n"
        string+="<data key=\"name\">%s</data>\n"%(self.name)
        string+="<data key=\"objectType\">%s</data>\n"%(self.kind)
        temp=""
        if(len(self.description)>0):
            for i in self.description:
                temp+=i+" "
            string+="<data key=\"description\">%s</data>\n"%(temp)
        temp=""
        if(len(self.comment_list)>0):
            for i in self.comment_list:
                temp+=(i+" ")
        for i in self.other_attributes.keys():
            for j in self.other_attributes[i]:
                temp+=(j+" ")
        if(len(temp)>0):
            string+="<data key=\"tags\">%s</data>\n"%(temp)
        string+="</node>\n"
        return string

In [4]:
def get_distinct_column_values(obj):
    ls=[]
    for i in obj:
        if(type(i)!=str):
            continue
        try:
            ls.index(i)
        except ValueError:
            ls.append(i)
    return ls

In [5]:
def initialise_level(identifiers,kind,names,parents,G,typ="Child",descriptions=[],dct={},comments=[]):
    l=len(identifiers)
    if(type(parents)==str):
        parent_obj=G.node[parents.strip()]["link"]
        for i in range(0,l):
            try:
                if(type(identifiers[i])!=str or type(names[i])!=str):
                    continue
                temp=node(identifiers[i],kind,names[i],G,parents)
                temp.insert_into_graph(G)
                if(len(descriptions)!=0):
                    temp.add_description(descriptions[i])
                if(len(dct.keys())!=0):
                    for key in dct.keys():
                        if(type(dct[key][i])==str):
                            temp.add_other_attribute(key,dct[key][i])
                if(len(comments)!=0):
                    try:
                        temp.add_comment(comments[i].values[0])
                    except KeyError:
                        ;
                    except AttributeError:
                        temp.add_comment(comments[i])
                parent_obj.add_child(temp,G,typ)
            except KeyError:
                ;
    else:
        for i in range(0,l):
            try:
                if(type(parents[i])!=str or type(identifiers[i])!=str or type(names[i])!=str):
                    continue
                parent_obj=G.node[parents[i].strip()]["link"]
                temp=node(identifiers[i],kind,names[i],G,parents[i])
                if(len(descriptions)!=0):
                    temp.add_description(descriptions[i])
                if(len(dct.keys())!=0):
                    for key in dct.keys():
                        if(type(dct[key][i])==str):
                            temp.add_other_attribute(key,dct[key][i])
                if(len(comments)!=0):
                    try:
                        temp.add_comment(comments[i].values[0])
                    except KeyError:
                        ;
                    except AttributeError:
                        temp.add_comment(comments[i])
                temp.insert_into_graph(G)
                parent_obj.add_child(temp,G,typ)
            except KeyError:
                ;

In [6]:
def add_dependent_relative(source,relative,G,typ="Pre-Requisite"):
    l=len(source)
    for i in range(0,l):
        if(type(relative[i])!=str):
            continue
        G.add_edge(relative[i].strip(),source[i].strip(),category=typ)

# KG_Maths

In [7]:
numeracy=node("Num","domain","numeracy",G)

In [8]:
temp=pd.read_csv("Funtoot/KG_Maths/ConceptList-Table 1.csv")
numeracy.insert_into_graph(G)
area_names=get_distinct_column_values(temp["Group"])
initialise_level(["AR","GE","CC","ST"],"areas",area_names,"numeracy",G)
initialise_level(temp["Concept Code"],"dimension",temp["Concepts in Mathematics"],temp["Group"],G)

In [9]:
temp=pd.read_csv("Funtoot/KG_Maths/ConceptDependency-Table 1.csv",header=1)
add_dependent_relative(temp["Concept"],temp["Dependency"],G)

In [10]:
temp=pd.read_csv("Funtoot/KG_Maths/LDs-Table 1.csv",header=1)
initialise_level(temp["LD Code"],"Misconception",temp["Error Category"],
                 "numeracy",G,"Induces",temp["Teacher / Parent Description"],dct={"Class Category":temp["Class Category"]})

# KG_(Rest)

In [11]:
#Get main domains
dir_list=[]
for root, dirs, files in os.walk("Funtoot", topdown=False):
    for name in dirs:
        if(name.find("KG")==0 and name.find("_Maths")==-1):
            dir_list.append(name[3:])

In [12]:
for dirname in dir_list:
    print (dirname)
    temp=pd.read_csv("Funtoot/KG_%s/Scope-Table 1.csv"%(dirname),header=1)
    ls=[]
    for i in range(0,len(temp)):
        ls.append("C%s"%(i))
    initialise_level(ls,"Concepts",temp["Subconcepts"].dropna(),dirname,G,"Child")
    ls=[]
    for i in range(0,len(temp)):
        ls.append("MC%s"%(i))
    dct={}
    for i in temp.keys():
        if(i.lower().find("grade")==0):
            dct["grade"]=temp[i]
        elif(i.lower().find("ld")==0):
            dct["ld"]=temp[i]
    comment=[]
    flag=False
    for i in temp.keys():
        if(temp[i].dtype!=object):
            flag=True
        if(flag and temp[i].dtype==object):
            comment.append(temp[i])
    try:
        comment = pd.concat(comment).dropna()
        initialise_level(ls,"MicroConcepts",temp["Sub Sub Concept"].dropna(),temp["Subconcepts"].dropna(),
                         G,"Child",temp["Scope Items"],dct,comment)
    except ValueError:
        initialise_level(ls,"MicroConcepts",temp["Sub Sub Concept"].dropna(),temp["Subconcepts"].dropna(),
                         G,"Child",temp["Scope Items"],dct)       
    temp=pd.read_csv("Funtoot/KG_%s/SubConceptDependency-Table 1.csv"%(dirname),header=1)
    add_dependent_relative(temp["Subconcept"],temp["Dependency"],G)

    temp=pd.read_csv("Funtoot/KG_%s/SSC Dependency-Table 1.csv"%(dirname),header=1)
    add_dependent_relative(temp["SSC"],temp["Dependency"],G)

    temp=pd.read_csv("Funtoot/KG_%s/LDs-Table 1.csv"%(dirname),header=1)
    try:
        initialise_level(temp["LD Code"],"Misconception",temp["Error Category"],temp["Sub Sub Concept"],G,
                         "Induces",temp["Teacher / Parent Description"],{"Example":temp["Example"],
                                                                         "Class Category":temp["Class Category"]})
    except KeyError:
        initialise_level(temp["LD Code"],"Misconception",temp["Error Category"],temp["Sub Sub Concept"],G,
                         "Induces",temp["Teacher / Parent Description"],{"Class Category":temp["Class Category"]})

Addition
Capacity
Data Handling
Division
Fractions
Length
Money
Multiplication
Number System
Patterns
Points and Lines
Shapes
Subtraction
Time
Triangles
Weight


In [13]:
def parse_edge(temp):
    string=""
    x=G.node[temp[0]]["link"].identifier
    y=G.node[temp[1]]["link"].identifier
    string+="<edge source=\"%s\" target=\"%s\">\n"%(x,y)
    string+="<data key=\"category\">%s</data>\n"%(G[temp[0]][temp[1]]['category'])
    string+="</edge>"
    return string

In [35]:
G.node["Introduction to triangles"]["link"].print_node()
G.node["Understanding triangles"]["link"].print_node()

Identifier: Num:GE:TS:C0 
Name: Introduction to triangles 
Kind: Concepts 
Parent: Triangles
Description: []
Comment List: []
Other Attributes: dict_keys([])
Identifier: Num:GE:TS:C0:MC0 
Name: Understanding triangles  
Kind: MicroConcepts 
Parent: Introduction to triangles
Description: ['Elements of triangles (vertex, interior angles, exterior angles)\nRegions of triangles\n']
Comment List: []
Other Attributes: dict_keys([])


In [14]:
"""for i in G.nodes():
    print(G.node[i]["link"].parse_graphML())"""

<node id="Num:AR:FC:C2:MC0:FC003">
<data key="label">Concept</data>
<data key="name">Denominator</data>
<data key="objectType">Misconception</data>
<data key="description">Learner is not aware of the concept of a denominator. </data>
<data key="tags">Introduction </data>
</node>

<node id="Num:GE:TS:C0">
<data key="label">Concept</data>
<data key="name">Introduction to triangles</data>
<data key="objectType">Concepts</data>
</node>

<node id="Num:AR:ML:C5:MC5:ML006">
<data key="label">Concept</data>
<data key="name">Application of associative property</data>
<data key="objectType">Misconception</data>
<data key="description">Learner may know the associative property of multiplication but fails to apply it to simplify the while multiplying.
Example: Learner able to find the product of three or more numbers, such as
8 × 13 × 5, because they fail to recognize that it is much easier to multiply the numbers in a different order. </data>
<data key="tags">Associative property of multiplicatio

In [23]:
"""for i in G.edges():
    print(parse_edge(i))"""

<edge source="Num:GE:TS:C0" target="Num:GE:TS:C0:MC0">
<data key="category">Child</data>
</edge>
<edge source="Num:GE:TS:C0" target="Num:GE:TS:C2">
<data key="category">Pre-Requisite</data>
</edge>
<edge source="Num:AR:AT:C11:MC10" target="Num:AR:FC:C34:MC30">
<data key="category">Pre-Requisite</data>
</edge>
<edge source="Num:GE:TS:C0:MC0" target="Num:GE:TS:C0:MC0:TS004">
<data key="category">Induces</data>
</edge>
<edge source="Num:GE:TS:C0:MC0" target="Num:GE:TS:C0:MC0:TS016">
<data key="category">Induces</data>
</edge>
<edge source="Num:GE:TS:C0:MC0" target="Num:GE:TS:C0:MC0:TS017">
<data key="category">Induces</data>
</edge>
<edge source="Num:GE:TS:C0:MC0" target="Num:GE:TS:C2:MC1">
<data key="category">Pre-Requisite</data>
</edge>
<edge source="Num:GE:TS:C0:MC0" target="Num:GE:TS:C0:MC0:TS015">
<data key="category">Induces</data>
</edge>
<edge source="Num:GE:TS:C0:MC0" target="Num:GE:TS:C0:MC0:TS014">
<data key="category">Induces</data>
</edge>
<edge source="Num:AR:FC:C18" target